<a href="https://colab.research.google.com/github/Niirav213/Early-warning-for-critical-events-Predict-things-like-sepsis-or-heart-failure-before-they-happen/blob/main/Sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from google.colab import files

print("Upload training_setA folder (zipped)")
uploaded_A = files.upload()  # Upload your training_setA.zip
print("Upload training_setB folder (zipped)")
uploaded_B = files.upload()  # Upload your training_setB.zip

# Unzip
import zipfile

# Get the actual filename from the uploaded dictionary
zip_file_A_name = list(uploaded_A.keys())[0]
zip_file_B_name = list(uploaded_B.keys())[0]


with zipfile.ZipFile(zip_file_A_name, 'r') as zip_ref:
    zip_ref.extractall("training_setA")

with zipfile.ZipFile(zip_file_B_name, 'r') as zip_ref:
    zip_ref.extractall("training_setB")

# List files in the extracted directories to help identify the correct path
#print("\nFiles in training_setA:")
#for root, dirs, files in os.walk("training_setA"):
 #   for name in files:
      #  print(os.path.join(root, name))

#print("\nFiles in training_setB:")
#for root, dirs, files in os.walk("training_setB"):
 #   for name in files:
 #       print(os.path.join(root, name))




Upload training_setA folder (zipped)


Upload training_setB folder (zipped)


IndexError: list index out of range

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# =========================
# BUILD PREPROCESSING PIPELINE
# =========================
def build_pipeline(df, target="SepsisLabel", missing_threshold=0.95):
    """
    Build and fit preprocessing pipeline on a GLOBAL dataframe.
    """

    # 1. Drop highly missing features (GLOBAL missingness)
    missing_frac = df.isnull().mean()
    features_to_drop = missing_frac[missing_frac > missing_threshold].index.tolist()
    df_reduced = df.drop(columns=features_to_drop)

    # 2. Separate numeric and categorical features
    num_features = (
        df_reduced
        .select_dtypes(include=["float64", "int64"])
        .drop(columns=[target])
        .columns
        .tolist()
    )

    cat_features = ["Gender", "Unit1", "Unit2"]
    cat_features = [c for c in cat_features if c in df_reduced.columns]
    num_features = [c for c in num_features if c not in cat_features]

    # 3. Transformers
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    # 4. ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_features),
            ("cat", categorical_transformer, cat_features)
        ]
    )

    # 5. Final pipeline
    pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

    # Fit pipeline on GLOBAL data
    X = df_reduced.drop(columns=[target])
    pipeline.fit(X)

    return pipeline


# =========================
# APPLY PIPELINE
# =========================
def apply_pipeline(df, pipeline, target="SepsisLabel"):
    X = df.drop(columns=[target])
    y = df[target]

    X_preprocessed = pipeline.transform(X)
    feature_names = pipeline.named_steps["preprocessor"].get_feature_names_out()

    X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=feature_names)

    return X_preprocessed_df, y


# =========================
# LOAD GLOBAL SAMPLE (KEY FIX)
# =========================
def load_global_sample(input_folder, max_files=200):
    """
    Load data from multiple patient files to estimate GLOBAL missingness.
    Prevents dropping labs absent in a single patient.
    """
    dfs = []
    count = 0

    for root, _, files in os.walk(input_folder):
        for f in files:
            if f.endswith(".psv"):
                df = pd.read_csv(os.path.join(root, f), sep="|")
                dfs.append(df)
                count += 1

                if count >= max_files:
                    return pd.concat(dfs, ignore_index=True)

    return pd.concat(dfs, ignore_index=True)


# =========================
# PROCESS DATASET FOLDER
# =========================
def main(input_folder, output_folder, target="SepsisLabel"):
    os.makedirs(output_folder, exist_ok=True)

    # Find all PSV files
    psv_files = []
    for root, _, files in os.walk(input_folder):
        for f in files:
            if f.endswith(".psv"):
                psv_files.append(os.path.join(root, f))

    if not psv_files:
        print(f"No PSV files found in {input_folder}")
        return

    print(f"Found {len(psv_files)} PSV files in {input_folder}")

    # 🔑 GLOBAL PIPELINE FIT (FIX)
    print("Building global sample for preprocessing...")
    global_df = load_global_sample(input_folder, max_files=200)
    print(f"Global sample shape: {global_df.shape}")

    pipeline = build_pipeline(global_df, target=target)

    # Process each patient file
    for file_path in psv_files:
        filename = os.path.basename(file_path)
        print(f"Processing {filename}...")

        df = pd.read_csv(file_path, sep="|")
        X_preprocessed, y = apply_pipeline(df, pipeline, target=target)

        df_out = pd.concat(
            [X_preprocessed, y.reset_index(drop=True)],
            axis=1
        )

        output_path = os.path.join(output_folder, filename)
        df_out.to_csv(output_path, sep="|", index=False)

    print(f"✅ Finished processing {input_folder}")


# =========================
# ENTRY POINT
# =========================
if __name__ == "__main__":

    # TRAINING (SET A)
    print("Processing training_setA...")
    main(
        input_folder="training_setA",
        output_folder="processed_training"
    )

    # VALIDATION / TEST (SET B)
    print("Processing training_setB...")
    main(
        input_folder="training_setB",
        output_folder="processed_training_setB"
    )


Streaming output truncated to the last 5000 lines.
Processing p101897.psv...
Processing p117141.psv...
Processing p111226.psv...
Processing p107399.psv...
Processing p119074.psv...
Processing p110014.psv...
Processing p104760.psv...
Processing p101315.psv...
Processing p118427.psv...
Processing p117763.psv...
Processing p104423.psv...
Processing p118198.psv...
Processing p107512.psv...
Processing p103042.psv...
Processing p106123.psv...
Processing p116996.psv...
Processing p103916.psv...
Processing p117431.psv...
Processing p117311.psv...
Processing p105372.psv...
Processing p100398.psv...
Processing p106184.psv...
Processing p100727.psv...
Processing p110701.psv...
Processing p108726.psv...
Processing p114630.psv...
Processing p118563.psv...
Processing p110720.psv...
Processing p119576.psv...
Processing p106469.psv...
Processing p112681.psv...
Processing p111831.psv...
Processing p116764.psv...
Processing p114499.psv...
Processing p115556.psv...
Processing p104538.psv...
Processing p1

In [ ]:
import os
import pandas as pd
import numpy as np


In [ ]:
import os
import pandas as pd

all_columns = set()

for f in os.listdir("processed_training"):
    if f.endswith(".psv"):
        df = pd.read_csv(os.path.join("processed_training", f), sep="|", nrows=1)
        all_columns.update(df.columns)

FEATURE_COLUMNS = sorted(c for c in all_columns if c != "SepsisLabel")

print("Final feature count:", len(FEATURE_COLUMNS))
print(FEATURE_COLUMNS)


Number of features: 32
['num__HR', 'num__O2Sat', 'num__Temp', 'num__SBP', 'num__MAP', 'num__DBP', 'num__Resp', 'num__BaseExcess', 'num__HCO3', 'num__FiO2', 'num__pH', 'num__PaCO2', 'num__BUN', 'num__Calcium', 'num__Chloride', 'num__Creatinine', 'num__Glucose', 'num__Lactate', 'num__Magnesium', 'num__Phosphate', 'num__Potassium', 'num__Hct', 'num__Hgb', 'num__PTT', 'num__WBC', 'num__Platelets', 'num__Age', 'num__HospAdmTime', 'num__ICULOS', 'cat__Gender_0', 'cat__Unit1_1', 'cat__Unit2_0']


In [ ]:
def load_patient_sequences(folder, feature_columns, early_hours=6):
    X, y = [], []

    for f in os.listdir(folder):
        df = pd.read_csv(os.path.join(folder, f), sep="|")

        # 🔑 FORCE SAME FEATURE SPACE
        df = df.reindex(columns=feature_columns + ["SepsisLabel"])
        df[feature_columns] = df[feature_columns].fillna(0.0)

        features = df[feature_columns].values
        labels = df["SepsisLabel"].values

        if labels.max() == 1:
            t_sepsis = np.where(labels == 1)[0][0]
            cutoff = t_sepsis - early_hours
            if cutoff <= 0:
                continue
            seq = features[:cutoff]
            label = 1
        else:
            seq = features
            label = 0

        X.append(seq)
        y.append(label)

    return X, np.array(y)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
def pad_data(X, max_len=None):
    if max_len is None:
        max_len = max(len(x) for x in X)

    X_pad = pad_sequences(
        X,
        maxlen=max_len,
        dtype="float32",
        padding="pre",
        truncating="pre"
    )

    return X_pad, max_len


In [ ]:
# TRAIN (Set A)
X_train_raw, y_train = load_patient_sequences("processed_training")
X_train, max_len = pad_data(X_train_raw)

# VALIDATION / TEST (Set B)
X_val_raw, y_val = load_patient_sequences("processed_training_setB")
X_val, _ = pad_data(X_val_raw, max_len=max_len)


In [ ]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

TIMESTEPS = X_train.shape[1]
N_FEATURES = X_train.shape[2]

print("TIMESTEPS:", TIMESTEPS)
print("N_FEATURES:", N_FEATURES)

assert N_FEATURES == 32


X_train shape: (19900, 336, 32)
X_val shape: (19658, 336, 15)
TIMESTEPS: 336
N_FEATURES: 32


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1]),
    y=y_train
)

class_weight = {0: class_weights[0], 1: class_weights[1]}
print(class_weight)


{0: np.float64(0.536503828318775), 1: np.float64(7.348596750369277)}


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

def build_lstm(units=64, dropout=0.3):
    model = Sequential([
        Input(shape=(TIMESTEPS, N_FEATURES)),
        LSTM(units, return_sequences=False),
        Dropout(dropout),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc")]
    )

    return model


In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    "units": [32, 64],
    "dropout": [0.3, 0.5]
}

best_auc = 0.0
best_model = None

for params in ParameterGrid(param_grid):
    print("Training with:", params)

    model = build_lstm(
        units=params["units"],
        dropout=params["dropout"]
    )

    model.fit(
        X_train,
        y_train,
        epochs=10,
        batch_size=32,
        class_weight=class_weight,
        validation_data=(X_val, y_val),
        verbose=1
    )

    val_auc = model.evaluate(X_val, y_val, verbose=0)[1]

    if val_auc > best_auc:
        best_auc = val_auc
        best_model = model


Training with: {'dropout': 0.3, 'units': 32}
Epoch 1/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - auc: 0.6124 - loss: 0.6797

ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 15 and 32 for '{{node sequential_1_1/lstm_1_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1_1/lstm_1_1/strided_slice_2, sequential_1_1/lstm_1_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [?,15], [32,128].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(None, 15), dtype=float32)
  • states=('tf.Tensor(shape=(None, 32), dtype=float32)', 'tf.Tensor(shape=(None, 32), dtype=float32)')
  • training=False

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
from sklearn.metrics import f1_score

y_prob = best_model.predict(X_val).ravel()

thresholds = np.linspace(0.1, 0.9, 50)
best_t, best_f1 = 0, 0

for t in thresholds:
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_val, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best threshold:", best_t)
print("Best F1:", best_f1)


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
y_pred = (y_prob > best_t).astype(int)

print("ROC-AUC:", roc_auc_score(y_val, y_prob))
print(classification_report(y_val, y_pred))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pos_weight = torch.tensor([scale_pos_weight]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

NameError: name 'scale_pos_weight' is not defined